In [1]:
"""
Purpose: To demonstrate how to use the 
skeletonization script

"""

'\nPurpose: To demonstrate how to use the \nskeletonization script\n\n'

# Helper Functions

In [2]:
from pykdtree.kdtree import KDTree
import time
import trimesh
import numpy as np

def filter_mesh_significant_outside_pieces(unfiltered_mesh,main_mesh,
                                           significance_threshold=2000,
                                           n_sample_points=3,
                                          print_flag=False):
    """
    Purpose; will take in a full, unfiltered mesh and find the biggest mesh piece, and then return a list of that mesh 
    with all of the other mesh fragments that are both above the significance_threshold AND outside of the biggest mesh piece

    Pseudocode: 
    1) split the meshes to unconnected pieces
    2) Filter the meshes for only those above the significance_threshold
    3) find the biggest mesh piece
    4) Iterate through all of the remaining pieces:
        a. Determine if mesh inside or outside main mesh
        b. If outside add to final list to return

    Returns: 
    1) list of significant mesh pieces that are not inside of main mesh

    """
    if print_flag:
        print("------Starting the mesh filter for significant outside pieces-------")

    mesh_pieces = unfiltered_mesh.split(only_watertight=False)
    
    if print_flag:
        print(f"There were {len(mesh_pieces)} pieces after mesh split")

    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    if print_flag:
        print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []



    final_mesh_pieces = []

    #final_mesh_pieces.append(main_mesh)
    for i,mesh in enumerate(significant_pieces):
        #get a random sample of points
        # points = np.array(mesh.vertices[:n_sample_points,:]) # OLD WAY OF DOING THIS
        idx = np.random.randint(len(mesh.vertices), size=n_sample_points)
        points = mesh.vertices[idx,:]


        start_time = time.time()
        signed_distance = trimesh.proximity.signed_distance(main_mesh,points)
        #print(f"Total time = {time.time() - start_time}")

        outside_percentage = sum(signed_distance <= 0)/n_sample_points
        if outside_percentage > 0.9:
            final_mesh_pieces.append(mesh)
            #print(f"Mesh piece {i} OUTSIDE mesh")
        else:
            #print(f"Mesh piece {i} inside mesh :( ")
            pass
                
    return final_mesh_pieces
    
def neuron_boolean_difference(main_mesh_verts,
                             main_mesh_faces,
                             child_mesh_verts,
                             child_mesh_faces,
                             distance_threshold=5,
                             significance_threshold=1000,
                             n_sample_points=3,
                             print_flag=False,
                             return_verts_faces=False):
    """
    returns the boolean difference of two meshes passed. Only returns meshes pieces that 
    are greater than the size threshold and outside of the main mesh
    
    Function operation: child_mesh - main_mesh
  

    Parameters: 
    main_mesh_verts (np.array): array of the reference mesh vertices
    main_mesh_faces (np.array): array of the reference mesh faces
    child_mesh_verts (np.array): array of the child mesh vertices that is to be compared to the main mesh
    child_mesh_faces (np.array): array of the child mesh faces that is to be compared to the main mesh
    
    Optional parameters:
    -- for controlling the mesh returned --
    distance_threshold (int) : distance away from the reference mesh that vertices from the child
                                mesh will considered distinct and part of the difference mesh 
                                (default=5)
    significance_threshold (int) : number of faces necessary for any distinct/unconnected part of the 
                                    difference mesh to be considered relevant and included in the difference mesh
                                (default=1000)
    n_sample_points (int) : number of vertices to check to see if submesh is a mesh located inside of the main mesh.
                            The less there are the quicker the speed for filtering the difference mesh
                            (default=3)
    
    
    Returns:
    difference_mesh_verts (np.array): array of vertices from the mesh boolean operation of child - main mesh
    difference_mesh_faces (np.array): array of faces from the mesh boolean operation of child - main mesh
    """
    
    #Create the kdtree from main mesh and run the queries
    
    import time
    global_time = time.time()
    start_time = time.time()
    mesh_tree = KDTree(main_mesh_verts)
    distances,closest_node = mesh_tree.query(child_mesh_verts)
    if print_flag:
        print(f"Total time for KDTree creation and queries: {time.time() - start_time}")
    
    if print_flag:
        print("Original number vertices in child mesh = " + str(len(child_mesh_verts)))
    vertex_indices = np.where(distances > distance_threshold)[0]
    if print_flag:
        print("Number of vertices after distance threshold applied =  " + str(len(vertex_indices)))
    
    start_time = time.time()
    set_vertex_indices = set(list(vertex_indices))
    face_indices_lookup = np.linspace(0,len(child_mesh_faces)-1,len(child_mesh_faces)).astype('int')
    face_indices_lookup_bool = [len(set_vertex_indices.intersection(set(tri))) > 0 for tri in child_mesh_faces]
    face_indices_lookup = face_indices_lookup[face_indices_lookup_bool]

    if print_flag:
        print(f"Total time for finding faces after distance threshold applied: {time.time() - start_time}")
    if len(child_mesh_verts)<=0 or len(child_mesh_faces)<=0 or len(face_indices_lookup)<= 0:
        print("inside boolean function and returning because child faces are 0")
        return []
    
    start_time = time.time()
    trimesh_original = trimesh.Trimesh(child_mesh_verts,child_mesh_faces,process=False) 
    new_submesh = trimesh_original.submesh([face_indices_lookup],only_watertight=False,append=True)
    
    pymesh_mesh = trimesh.Trimesh(main_mesh_verts,main_mesh_faces)
    #return new_submesh
    #filter the mesh for only significant pieces on the outside
    returned_mesh = filter_mesh_significant_outside_pieces(new_submesh,pymesh_mesh,
                                                           significance_threshold,
                                                           n_sample_points=n_sample_points,
                                                          print_flag=print_flag)
    
    if return_verts_faces == False:
        return returned_mesh
    else:
        total_returned_mesh = trimesh.Trimesh(vertices = np.array([]),
                                         faces = np.array([]))
        for r in returned_mesh:
            total_returned_mesh = total_returned_mesh + r

        if print_flag:
            print(f"Total time for filtering: {time.time() - start_time}")

        if print_flag:
            print(f"Total time for boolean difference: {time.time() - global_time}")
        return total_returned_mesh.vertices,total_returned_mesh.faces

    
    

In [3]:
import os, contextlib
import pathlib
import subprocess

def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    command_to_run = 'xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command
    #command_to_run = 'meshlabserver ' + script_command
    
    print(command_to_run)
    subprocess_result = subprocess.run(command_to_run,shell=True)
    
    return subprocess_result

def meshlab_fix_manifold_path_specific_mls(input_path_and_filename,
                                           output_path_and_filename="",
                                           segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if input_path_and_filename[-4:] == ".off":
        path_and_filename = input_path_and_filename[:-4]
        input_mesh = input_path_and_filename
    else:
        raise Exception("Not passed off file")
    
    
    if output_path_and_filename == "":
        output_mesh = path_and_filename+"_mls.off"
    else:
        output_mesh = output_path_and_filename
    
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    #print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [4]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(vertices=[], 
                                triangles=[],
                                neuron_ID="None",
                                folder="None",
                               path_and_filename="-1"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    if path_and_filename == "-1":
        #get the current file location
        if folder == "None":
            file_loc = pathlib.Path.cwd()
            
        else:
            file_loc = pathlib.Path.cwd() / folder
            
        filename = "neuron_" + str(neuron_ID)
        path_and_filename = file_loc / filename
    
    #print("path_and_filename = " + str(path_and_filename))
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    #print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename)#,str(filename),str(file_loc)

In [16]:
def recursive_skeletonizing(verts,faces,current_name,
                            directory,current_base_path,mesh_base_path,folder_name,
                           boolean_distance_threshold=1000,
                            boolean_significance_threshold=1000,
                            boolean_n_sample_points=10,
                            boolean_print_flag = False,
                           largest_mesh_significant=3000):
    

    current_layer = 1
    
    print(f"\n----------Starting {current_name}----------")
    global_time = time.time()
    #print out the off file
    initial_output_path = mesh_base_path + current_name
    
    write_Whole_Neuron_Off_file(vertices=verts, 
                                triangles=faces,
                               path_and_filename=initial_output_path)
    
    
    # do the poisson surface reconstruction
    
    script_name = "poisson_working_meshlab.mls"
    
    input_file_base = initial_output_path
    output_file = input_file_base + "_poisson"
    meshlab_script_path_and_name =current_base_path + script_name

    start_time = time.time()
    print("     Starting Screened Poisson")
    meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_file_base + ".off",
                                                       output_path_and_filename=output_file + ".off",
                                                     meshlab_script=meshlab_script_path_and_name)
    print(f"     Total_time for Screended Poisson = {time.time() - start_time}")
    
    #2) Filter away for largest_poisson_piece:
    
    new_mesh = trimesh.load_mesh(output_file + ".off")
    mesh_splits = new_mesh.split(only_watertight=False)
    mesh_lengths = np.array([len(split.faces) for split in mesh_splits])
    largest_index = np.where(mesh_lengths == np.max(mesh_lengths))
    largest_mesh = mesh_splits[largest_index][0]
    
    print("len(largest_mesh.vertices) = " + str(len(largest_mesh.vertices)))
    #4) If not of a significant size then return (Add the largest_poisson_piece to the Surface_Reconstruction_list and say that it was returned)
    if len(largest_mesh.vertices) < largest_mesh_significant:
        print(current_name + " returned because largest mesh not significant")
        return
    
    #5) If significant size output the mesh
    largest_mesh_path = mesh_base_path + current_name + "-0"
    write_Whole_Neuron_Off_file(vertices=largest_mesh.vertices, 
                                triangles=largest_mesh.faces,
                               path_and_filename=largest_mesh_path)
    
    
    #6) Run skeletonization on it:
    start_time = time.time()
    print("     Starting Calcification")
    cm.calcification(largest_mesh_path)
    print(f"     Total_time for Calcification = {time.time() - start_time}")
    
    
    #7) Subtract the largest_poisson_piece from leftover
    start_time = time.time()
    print("     Starting Mesh boolean difference")
    mesh_pieces =neuron_boolean_difference( largest_mesh.vertices,
                                               largest_mesh.faces,
                                                verts,
                                               faces,
                                               distance_threshold=boolean_distance_threshold,
                                               significance_threshold=boolean_significance_threshold,
                                               n_sample_points=boolean_n_sample_points,
                                              print_flag = boolean_print_flag)
    print(f"     Total_time for Mesh boolean difference = {time.time() - start_time}")
    
    print(f"Total time for one mesh piece skeleton = {time.time() - global_time}")

    print(f"{current_name} there were {len(mesh_pieces)} significant pieces leftover after largest mesh")
    
    if len(mesh_pieces) <= 0:
        print(f"{current_name} returning because 0 significant pieces")
        return 
    
    for j,piece in enumerate(mesh_pieces):
        new_current_name = current_name + "-" + str(j)
        recursive_skeletonizing(verts = piece.vertices,faces=piece.faces,current_name=new_current_name,
                               directory=directory,current_base_path=current_base_path,
                                mesh_base_path=mesh_base_path,folder_name=folder_name,
                               boolean_distance_threshold=boolean_distance_threshold,
                                boolean_significance_threshold=boolean_significance_threshold,
                                boolean_n_sample_points=boolean_n_sample_points,
                                boolean_print_flag = boolean_print_flag,
                               largest_mesh_significant=largest_mesh_significant)
    
    return

In [17]:
#read in the skeleton files into an array
def read_skeleton_revised(file_path):
    with open(file_path) as f:
        bones = np.array([])
        for line in f.readlines():
            #print(line)
            line = (np.array(line.split()[1:], float).reshape(-1, 3))
            #print(line[:-1])
            #print(line[1:])

            #print(bones.size)
            if bones.size <= 0:
                bones = np.stack((line[:-1],line[1:]),axis=1)
            else:
                bones = np.vstack((bones,(np.stack((line[:-1],line[1:]),axis=1))))
            #print(bones)


    return np.array(bones).astype(float)

In [18]:
import time
import numpy as np
import datajoint as dj
import networkx as nx

import matplotlib.pyplot as plt #for graph visualization
from scipy.spatial import distance_matrix
from tqdm import tqdm

def stitch_skeleton_with_degree_check_vp2(staring_edges,max_stitch_distance=18000,end_node=False):
    

    #unpacks so just list of vertices
    vertices_unpacked  = staring_edges.reshape(-1,3)

    #reduce the number of repeat vertices and convert to list
    unique_rows = np.unique(vertices_unpacked, axis=0)
    unique_rows_list = unique_rows.tolist()

    #assigns the number to the vertex (in the original vertex list) that corresponds to the index in the unique list
    vertices_unpacked_coefficients = [unique_rows_list.index(a) for a in vertices_unpacked.tolist()]

    #reshapes the vertex list to become an edge list
    edges_with_coefficients =  np.array(vertices_unpacked_coefficients).reshape(-1,2)
    
    #create the graph from the edges
    B = nx.Graph()
    B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_rows_list)])
    B.add_edges_from(edges_with_coefficients)

    # find the shortest distance between the two different subgraphs:
    from scipy.spatial import distance_matrix

    UG = B.to_undirected()

    # extract subgraphs
    sub_graphs = nx.connected_component_subgraphs(UG)

    subgraphs_list = []
    for i, sg in enumerate(sub_graphs):
        #print("subgraph {} has {} nodes".format(i, sg.number_of_nodes()))
        #print("\tNodes:", sg.nodes(data=True))
        #print("\tEdges:", sg.edges())
        subgraphs_list.append(sg)

    print("len_subgraphs AT BEGINNING = " + str(len(subgraphs_list)))

    while len(subgraphs_list) > 1:
#         if subgraph_counter>=len(subgraphs_list):
#             print("Breaking because the sugraph counter was larger than list")
#             break
#         print("subgraph_counter = " + str(subgraph_counter))
        current_sub_graph = subgraphs_list[0]
        #get all of the coordinates of this subgraph 
        #get all of the coordinates
        if end_node == True:
            current_nodes = np.array([x for x,n in current_sub_graph.degree() if n <= 1])
            current_coord = nx.get_node_attributes(current_sub_graph,'coordinates')
            current_coord_filter = { k:v for k,v in current_coord.items() if k in  current_nodes}
            current_coordinates = np.array(list(current_coord_filter.values()))
        
        else:
            #get all of the coordinates
            current_nodes = np.array(list(current_sub_graph.nodes))
            current_coord = nx.get_node_attributes(current_sub_graph,'coordinates')
            current_coordinates = np.array(list(current_coord.values()))
            
#         coord = nx.get_node_attributes(current_sub_graph,'coordinates')
#         current_nodes = np.array(list(current_sub_graph.nodes))
#         current_coordinates = np.array(list(coord.values()))

        #store the minimum distance
        min_dist = np.inf
        min_dist_subgraph_index = -1
        min_dist_edge = [-1,-1]
        min_dist_edge_index = [-1,-1]

        #i = 0
        for i, sg in enumerate(subgraphs_list):
            if i == 0:
                continue

            #get all of the coordinates
            if end_node == True:
                new_nodes = np.array([x for x,n in sg.degree() if n <= 1])
                #print("new nodes = " + str(new_nodes))
                new_coord = nx.get_node_attributes(sg,'coordinates')
                #print("new_coord = " + str(new_coord))
                new_coord_filter = { k:v for k,v in new_coord.items() if k in  new_nodes}
                new_coordinates = np.array(list(new_coord_filter.values()))
            else:
                #get all of the coordinates
                new_nodes = np.array(list(sg.nodes))
                new_coord = nx.get_node_attributes(sg,'coordinates')
                new_coordinates = np.array(list(new_coord.values()))
                
            #find the closest distance between those points
            a_New = current_coordinates
            b_New = new_coordinates

            # print("starting distance matrix")
            start_time = time.time()

            a_b_distance = distance_matrix(a_New, b_New)
            #print(a_b_distance)

            current_min = np.min(a_b_distance)
            dist_matrix_index = np.where(a_b_distance == current_min)
            min_indexes = dist_matrix_index[0][0],dist_matrix_index[1][0]

            #print("Min indexes = " + str(min_indexes))

            if current_min<min_dist:
                min_dist = current_min
                min_dist_subgraph_index = i
                min_dist_edge_index = [current_nodes[min_indexes[0]],
                                       new_nodes[min_indexes[1]]]
                min_dist_edge = [current_coordinates[min_indexes[0]],
                                        new_coordinates[min_indexes[1]]]
            #i += 1

        #add the shortest connection to the overall edges
        print("min_dist = " + str(min_dist))
        print("min_dist_subgraph_index = " + str(min_dist_subgraph_index))
        print("min_dist_edge_index = " + str(min_dist_edge_index))
        print("min_dist_edge = " + str(min_dist_edge))
        
        #check if the distance is too far:
        if min_dist > max_stitch_distance:
            print("********** breaking because min distance is too large*****")
            break
           
        

        #adds the new edge to the graph
        B.add_edge(*min_dist_edge_index)

        #
        #print("edges after adding")
        #print(B.edges())
        UG = B.to_undirected()

        # extract subgraphs
        sub_graphs = nx.connected_component_subgraphs(UG)
        subgraphs_list = []
        for i, sg in enumerate(sub_graphs):
#             print("subgraph {} has {} nodes".format(i, sg.number_of_nodes()))
#             print("\tNodes:", sg.nodes(data=True))
#             print("\tEdges:", sg.edges())
            subgraphs_list.append(sg)

        print("len_subgraphs AT END= " + str(len(subgraphs_list)))
        
        
    # get the largest subgraph!!! in case have missing pieces
    
    #add all the new edges to the 
    Gc = max(nx.connected_component_subgraphs(B), key=len)
    
    total_coord = nx.get_node_attributes(B,'coordinates')
    #print("len(total_coord)= " + str(len(total_coord)))
    current_coordinates = np.array(list(total_coord.values()))
    #print("current_coordinates.shape= " + str(current_coordinates.shape))
    #print("Gc.edges() = " + str(Gc.edges()))
    return current_coordinates[Gc.edges()]


"""
For finding the distances of the skeleton
"""
def find_skeleton_distance(example_edges):
    total_distance = np.sum([np.linalg.norm(a-b) for a,b in example_edges])
    return total_distance

from scipy.spatial import distance

def find_skeleton_distance_scipy(example_edges):
    total_distance = np.sum([distance.euclidean(a, b) for a,b in example_edges])
    return total_distance

# Main Function 

In [19]:
import calcification_Module as cm
import time
import os

def calculate_skeleton_edges(input_verts,input_faces,segment_id,
                            boolean_distance_threshold=1000,
                                   boolean_significance_threshold=300,
                                   boolean_n_sample_points=10,
                                   boolean_print_flag=False,
                                   largest_mesh_significant=300):
    original_mesh = trimesh.Trimesh(vertices=input_verts,faces=input_faces)
    name2 = segment_id
    
    
    directory = "./" + name2
    if not os.path.exists(directory):
        os.makedirs(directory)


    current_base_path = str(pathlib.Path.cwd()) + "/"
    mesh_base_path = current_base_path + str(name2) + "/"
    folder_name = str(name2)

    current_layer = 1
    
    recursive_skeletonizing(original_mesh.vertices,original_mesh.faces,name2,
                            directory,current_base_path,mesh_base_path,folder_name,
                        boolean_distance_threshold=boolean_distance_threshold,
                            boolean_significance_threshold=boolean_significance_threshold,
                            boolean_n_sample_points=boolean_n_sample_points,
                            boolean_print_flag = boolean_print_flag,
                           largest_mesh_significant=largest_mesh_significant)
    
    cgal_skeleton_file_list = []
    
    for file in os.listdir(name2):
        if file.endswith(".cgal"):
            cgal_skeleton_file_list.append(str(os.path.join(name2, file)))
    if len(cgal_skeleton_file_list) <= 0:
        #check to see if cgal list is empty
        raise Exception("There were no cgal files generated in process")
    
    bone_array_total = np.vstack([read_skeleton_revised(k) 
                                  for k in cgal_skeleton_file_list])

    unique_skeleton_verts = bone_array_total.reshape(-1,3)
    edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)


    total_edges = np.array([])

    for k in cgal_skeleton_file_list:
        bone_array = read_skeleton_revised(k) 

        #add the skeleton edges to the total edges
        if not total_edges.any():
            total_edges = bone_array
        else:
            total_edges = np.vstack([total_edges,bone_array])

    total_edges_stitched = stitch_skeleton_with_degree_check_vp2(total_edges)
    
    unique_skeleton_verts = total_edges_stitched.reshape(-1,3)
    edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)
    
    #erase the folder with all of the data in it
    import shutil
    shutil.rmtree(directory)
    
    #return the edges
    return edges,unique_skeleton_verts


    


# Function call to skeletonize a mesh with vertices and faces

In [20]:
import trimesh

main_mesh = trimesh.load("107816118160698192_original_decimated.off")
input_verts = main_mesh.vertices
input_faces = main_mesh.faces
segment_id = "107816118160698192_test"
edges,unique_skeleton_verts = calculate_skeleton_edges(input_verts,input_faces,segment_id,
                                   boolean_distance_threshold=1000,
                                   boolean_significance_threshold=300,
                                   boolean_n_sample_points=10,
                                   boolean_print_flag=False,
                                   largest_mesh_significant=300)


----------Starting 107816118160698192_test----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/107816118160698192_test/107816118160698192_test.off -o /notebooks/Platinum_Blender/107816118160698192_test/107816118160698192_test_poisson.off -s /notebooks/Platinum_Blender/poisson_working_meshlab.mls
     Total_time for Screended Poisson = 46.34484839439392


face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

len(largest_mesh.vertices) = 170930
     Starting Calcification
     Total_time for Calcification = 10.533742189407349
     Starting Mesh boolean difference
     Total_time for Mesh boolean difference = 23.41098642349243
Total time for one mesh piece skeleton = 90.61589312553406
107816118160698192_test there were 38 significant pieces leftover after largest mesh

----------Starting 107816118160698192_test-0----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/107816118160698192_test/107816118160698192_test-0.off -o /notebooks/Platinum_Blender/107816118160698192_test/107816118160698192_test-0_poisson.off -s /notebooks/Platinum_Blender/poisson_working_meshlab.mls


face_normals all zero, ignoring!


     Total_time for Screended Poisson = 5.578112602233887
len(largest_mesh.vertices) = 914
     Starting Calcification
     Total_time for Calcification = 0.08797025680541992
     Starting Mesh boolean difference
     Total_time for Mesh boolean difference = 0.05837607383728027
Total time for one mesh piece skeleton = 5.814083576202393
107816118160698192_test-0 there were 2 significant pieces leftover after largest mesh

----------Starting 107816118160698192_test-0-0----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/107816118160698192_test/107816118160698192_test-0-0.off -o /notebooks/Platinum_Blender/107816118160698192_test/107816118160698192_test-0-0_poisson.off -s /notebooks/Platinum_Blender/poisson_working_meshlab.mls
     Total_time for Screended Poisson = 5.784937143325806
len(largest_mesh.vertices) = 366
     Starting Calcification
     Total_time for Calcification = 0.053861141204833984
     Starting 

...

     Total_time for Screended Poisson = 5.6711766719818115
len(largest_mesh.vertices) = 1642
     Starting Calcification
     Total_time for Calcification = 0.10748744010925293
     Starting Mesh boolean difference
     Total_time for Mesh boolean difference = 0.11571669578552246
Total time for one mesh piece skeleton = 6.0357441902160645
107816118160698192_test-36 there were 1 significant pieces leftover after largest mesh

----------Starting 107816118160698192_test-36-0----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/107816118160698192_test/107816118160698192_test-36-0.off -o /notebooks/Platinum_Blender/107816118160698192_test/107816118160698192_test-36-0_poisson.off -s /notebooks/Platinum_Blender/poisson_working_meshlab.mls


face_normals all zero, ignoring!


     Total_time for Screended Poisson = 5.404125690460205
len(largest_mesh.vertices) = 708
     Starting Calcification
     Total_time for Calcification = 0.06995820999145508
     Starting Mesh boolean difference
     Total_time for Mesh boolean difference = 0.0426025390625
Total time for one mesh piece skeleton = 5.592268466949463
107816118160698192_test-36-0 there were 1 significant pieces leftover after largest mesh

----------Starting 107816118160698192_test-36-0-0----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/107816118160698192_test/107816118160698192_test-36-0-0.off -o /notebooks/Platinum_Blender/107816118160698192_test/107816118160698192_test-36-0-0_poisson.off -s /notebooks/Platinum_Blender/poisson_working_meshlab.mls
     Total_time for Screended Poisson = 5.553109407424927
len(largest_mesh.vertices) = 1458
     Starting Calcification
     Total_time for Calcification = 0.1406569480895996
     St

face_normals all zero, ignoring!


     Total_time for Screended Poisson = 5.460998296737671
len(largest_mesh.vertices) = 1212
     Starting Calcification
     Total_time for Calcification = 0.08656501770019531
     Starting Mesh boolean difference
inside boolean function and returning because child faces are 0
     Total_time for Mesh boolean difference = 0.0022695064544677734
Total time for one mesh piece skeleton = 5.609481334686279
107816118160698192_test-37 there were 0 significant pieces leftover after largest mesh
107816118160698192_test-37 returning because 0 significant pieces
len_subgraphs AT BEGINNING = 69
min_dist = 2515.5182368649207
min_dist_subgraph_index = 2
min_dist_edge_index = [199, 230]
min_dist_edge = [array([1219010.,  653874.,  891310.]), array([1220750.,  655668.,  891024.])]
len_subgraphs AT END= 68
min_dist = 1095.8727115865236
min_dist_subgraph_index = 22
min_dist_edge_index = [3557, 3654]
min_dist_edge = [array([1261670.,  725623.,  869582.]), array([1262480.,  725257.,  868941.])]
len_subgra

In [21]:
# test that the skeletonization went well
import ipyvolume as ipv
ipv.figure()
# we draw the tetrahedron
original_mesh = trimesh.load_mesh("107816118160698192_original_decimated.off")


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))
